Implement svc model

In [6]:

import sys
#


#imports for models
import svc
import mlp
import data_loader

#imports for Anchor
from __future__ import print_function
import numpy as np
np.random.seed(1)
import sys

%load_ext autoreload
%autoreload 2

from anchor import anchor_tabular

import csv

dl = data_loader.DataLoader()
dl.load(path='../data/')
dl.load_sample_data("../data/sample_data.pkl")
svc_model = svc.SVC('../src/MITx-SVC.pkl', from_file=True)

mlp_model = mlp.MLPClassifier('C:/Users/Thomas/Desktop/Explainable-Ai-Comps-2024/MOOC/src/MITx-MLP.pkl', from_file=True)

#implementing ANCHOR

#list of feature titles for anchor to use to make explanations
MOOC_feature_names = ['viewed', 'gender',  'grade','nevents', 'ndays_act', 'nplay_video',  'nchapters',    'age',  'votes',    'num_words']

#names for the categorical features
MOOC_categorical_names = {
    0: ['not viewed', 'viewed'],
    1: ['f', 'm'] #swapped 2/5    
}

np.random.seed(55057)




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users/Thomas/Desktop/Explainable-Ai-Comps-2024/mooc/src\data_loader.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['gender']] = integer_encoded
C:\Users\Thomas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Thomas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\P

In [ ]:

explainer = anchor_tabular.AnchorTabularExplainer(
    ['fail','pass'], #class names
    MOOC_feature_names, #feature names
    dl.X_train, #training data
    MOOC_categorical_names #categorical names)
)

file_path = 'output-test.csv'
ds = dl.X_incomplete #data set being explained

headers = ['index'] + MOOC_feature_names + ['true class', 'SVM prediction', 'SVM anchor', 'SVM precision', 'SVM coverage', 'MLP prediction', 'MLP anchor', 'MLP precision', 'MLP coverage']

with open(file_path, 'w', newline='') as file:
   writer = csv.writer(file)
   writer.writerow(headers)

data = []

indicies = range(0,50) #indicies of the data set to explain


        

In [ ]:
for i in indicies:
    print(dl.X_incomplete[i])
    pred_svm = ['fail','pass'][svc_model.predict(ds[i].reshape(1, -1))[0]]
    pred_mlp = ['fail', 'pass'][mlp_model.predict(ds[i].reshape(1, -1))[0]]
    print(pred_svm, pred_mlp)
    
    try:
      exp_svm = explainer.explain_instance(ds[i], svc_model.predict, threshold=0.99)
      exp_mlp = explainer.explain_instance(ds[i], mlp_model.predict, threshold=0.99)
    except:
      print ("Error: likley querried beyond list")

    #print('Anchor: %s' % (' AND '.join(exp.names())))
    anchor_svm = ' AND '.join(exp_svm.names())
    anchor_mlp = ' AND '.join(exp_mlp.names())

    precision_svm, coverage_svm = exp_svm.precision(), exp_svm.coverage()
    precision_mlp, coverage_mlp = exp_mlp.precision(), exp_mlp.coverage()

    true_class = 'incomplete' 

    data.append([i] + list(ds[i]) + [true_class, pred_svm, anchor_svm, precision_svm, coverage_svm, pred_mlp, anchor_mlp, precision_mlp, coverage_mlp])

    if i % 1 == 0:
        print(i)
        

        # print('\tTrue class: %s' % true_class)
        # print('\tSVM prediction: %s' % pred_svm)
        # print('\tSVM anchor: %s' % anchor_svm)
        # print('\tSVM precision: %.2f' % precision_svm)
        # print('\tSVM coverage: %.2f' % coverage_svm)
        # print('\tMLP prediction: %s' % pred_mlp)
        # print('\tMLP anchor: %s' % anchor_mlp)
        # print('\tMLP precision: %.2f' % precision_mlp)
        # print('\tMLP coverage: %.2f' % coverage_mlp)
        print('')
        with open(file_path, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(data)
        data = []